<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-3/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import tensorflow as tf
import keras as ks
import random
import csv

from tensorflow.keras.layers import Dense, Input,InputLayer,Flatten,Activation,LSTM,SimpleRNN,GRU,TimeDistributed,Embedding
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
random.seed(hash("seriously you compete with me") % 2**32 - 1)
np.random.seed(hash("i am mohammed safi") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)

In [3]:
from tensorflow.python.keras import backend as K
K._get_available_gpus()

['/device:GPU:0']

In [4]:
!nvidia-smi

Sat Apr 23 08:54:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W /  70W |    264MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb
import wandb

from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8 MB 8.5 MB/s 
     |████████████████████████████████| 181 kB 68.7 MB/s 
     |████████████████████████████████| 144 kB 71.6 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=a76af9e5c518468fa64bc5295100472fef4d028568a1d8e18895890151bdc65e
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# English as source language and Telugu as target language
source_language='en'
target_language='te'

In [12]:
def load_data():
    path = r"/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/te/lexicons/"

    train_file_path = path + "te.translit.sampled.train.tsv"
    val_file_path = path + "te.translit.sampled.dev.tsv"

    

    path = r"/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/te/lexicons/"

    train_file_path = path + "te.translit.sampled.train.tsv"
    val_file_path = path + "te.translit.sampled.dev.tsv"

    train_tsv_file = open(train_file_path, "r", encoding="utf-8")
    train_dataset = csv.reader(train_tsv_file, delimiter="\t")

    val_tsv_file = open(val_file_path, "r", encoding="utf-8")
    val_dataset = csv.reader(val_tsv_file, delimiter="\t")

    return train_dataset, val_dataset


In [7]:
def prepare_data():

    train_dataset, val_dataset = load_data()
    
    input = []
    target = []
    #print(english)
    for i in train_dataset:
        #print(i) 
        target.append(i[0])
        #print(i[1])
        input.append(i[1])
    #print(english)
    target = np.array(target)
    input = np.array(input)

    # Validation data
    val_input = []
    val_target = []

    for i in val_dataset:
        val_target.append(i[0])
        val_input.append(i[1])

    val_target = np.array(val_target)
    val_input = np.array(val_input)

    for i in range(len(target)):
        target[i] = "\t" + target[i] + "\n"
    
    for i in range(len(val_target)):
        val_target[i] = "\t" + val_target[i] + "\n"

    return input, target, val_input, val_target



In [8]:
def getTokens(input, target, val_input, val_target):
    # Getting input and target language characters

    # Training set
    input_tokens = set()
    target_tokens = set()

    for word in input:
        for char in word:
            if char not in input_tokens:
                input_tokens.add(char)

    for word in target:
        for char in word:
            if char not in target_tokens:
                target_tokens.add(char)

    # Validation set
    val_input_tokens = set()
    val_target_tokens = set()

    for word in val_input:
        for char in word:
            if char not in val_input_tokens:
                val_input_tokens.add(char)

    for word in val_target:
        for char in word:
            if char not in val_target_tokens:
                val_target_tokens.add(char)

    input_tokens = sorted(list(input_tokens))
    target_tokens = sorted(list(target_tokens))
    
    return input_tokens, target_tokens, val_input_tokens, val_target_tokens


In [13]:
input, target, val_input, val_target = prepare_data()
input_tokens, target_tokens, val_input_tokens, val_target_tokens = getTokens(input, target, val_input, val_target)


In [14]:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(txt) for txt in input])
max_decoder_seq_length = max([len(txt) for txt in target])

input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

print("Number of data points:", len(input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of data points: 58550
Number of unique input tokens: 26
Number of unique output tokens: 65
Max sequence length for inputs: 25
Max sequence length for outputs: 20


ENCODER AND DECODER INPUTS

In [15]:
def one_hot_encoding(input, target, val_input, val_target, input_tokens, target_tokens):

    input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])

    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

    
    encoder_input_data = np.zeros(
        (len(input), max_encoder_seq_length), dtype = "float32" )
    decoder_input_data = np.zeros(
        (len(input), max_decoder_seq_length), dtype = "float32" )
    decoder_target_data = np.zeros(
        (len(input), max_decoder_seq_length, num_decoder_tokens), dtype = "float32" )
    
    for i, (inp, tar) in enumerate(zip(input, target)):
        for t, char in enumerate(inp):
            encoder_input_data[i,t] = input_token_index[char]

        for t, char in enumerate(tar):
            decoder_input_data[i,t] = target_token_index[char]
            if t>0:
                decoder_target_data[i,t-1,target_token_index[char]] = 1.0

    

    encoder_val_input_data = np.zeros(
        (len(val_input), max_encoder_seq_length), dtype = "float32" )
    decoder_val_input_data = np.zeros(
        (len(val_input), max_decoder_seq_length), dtype = "float32" )
    decoder_val_target_data = np.zeros(
        (len(val_input), max_decoder_seq_length, num_decoder_tokens), dtype = "float32" )

    for i, (inp, tar) in enumerate(zip(val_input, val_target)):
        for t, char in enumerate(inp):
            encoder_val_input_data[i,t] = input_token_index[char]

        for t, char in enumerate(tar):
            decoder_val_input_data[i,t] = target_token_index[char]
            if t>0:
                decoder_target_data[i,t-1,target_token_index[char]] = 1.0

    return encoder_input_data, decoder_input_data, decoder_target_data, encoder_val_input_data, decoder_val_input_data, decoder_val_target_data


In [16]:
encoder_input_data, decoder_input_data, decoder_target_data, encoder_val_input_data, decoder_val_input_data, decoder_val_target_data =one_hot_encoding(input, target, val_input, val_target, input_tokens, target_tokens)

In [17]:
encoder_input_data.shape

(58550, 25)

In [21]:
def rnn( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder

  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)
  #enc_out = enc_in

  enc_layers = []
  enc_states = []
  # for i in range(1,num_encoders+1):
  #   encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   enc_layers.append(encoder)
  #   enc_out, state_h = encoder(enc_out)
  #   enc_states.append([state_h])
  encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  enc_layers.append(encoder)
  enc_out, state_h = encoder(enc_out)
  enc_states.append([state_h])
  
  if num_encoders>1:
      encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
      enc_layers.append(encoder)
      enc_out, state_h2 = encoder(enc_out)
      enc_states.append([state_h2])
  
  if num_encoders>2:
      encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
      enc_layers.append(encoder)
      enc_out, state_h3 = encoder(enc_out)
      enc_states.append([state_h3])
  

  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True, mask_zero = True)(dec_in)
  #dec_out = dec_in

  dec_layers = []
  # for i in range(1,num_decoders+1):
  #   decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  #   dec_layers.append(decoder)
  #   dec_out, _ = decoder(dec_out, initial_state = enc_states[0])
  decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
  dec_layers.append(decoder)
  dec_out, _ = decoder(dec_out, initial_state = enc_states[0])

  if num_decoders>1:
      decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
      dec_layers.append(decoder)
      dec_out, _ = decoder(dec_out, initial_state = enc_states[1])
  
  if num_decoders>2:
      decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
      dec_layers.append(decoder)
      dec_out, _ = decoder(dec_out, initial_state = enc_states[2])
    
  dec_dense =TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def lstm( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  
  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []
  for i in range(1,num_encoders+1):
    encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    enc_layers.append(encoder)
    enc_out, state_h, state_c = encoder(enc_out)
    enc_states.append([state_h, state_c])
    

  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []
  for i in range(1,num_decoders+1):
    decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    dec_layers.append(decoder)
    dec_out, _, _ = decoder(dec_out, initial_state = enc_states[0])
  
  
  dec_dense = Dense(num_decoder_tokens, activation="softmax")
  dec_out = dec_dense(dec_out)
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def gru(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  
 
  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []
  for i in range(1,num_encoders+1):
    encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    enc_layers.append(encoder)
    enc_out, state_h = encoder(enc_out)
    enc_states.append([state_h])
    

  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []
  for i in range(1,num_decoders+1):
    decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
    dec_layers.append(decoder)
    dec_out, _ = decoder(dec_out, initial_state = enc_states[0])
  
  
  dec_dense = Dense(num_decoder_tokens, activation="softmax")
  dec_out = dec_dense(dec_out)
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [19]:
def transliteration(config):
  num_encoders=config["num_encoders"]
  cell=config["cell"]
  embed_size=config["embed_size"]
  dropout=config["dropout"]
  num_decoders=config["num_decoders"]
  hidden_layer_size=config["hidden_layer_size"]

  if cell == "RNN":
    model, enc_layers, dec_layers=rnn( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  elif cell == "LSTM":
    model, enc_layers, dec_layers=lstm(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  elif cell == "GRU":
    model, enc_layers, dec_layers=gru(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  


In [22]:

batch_size = 128        
epochs = 7             
input_embedding_size = 512
hidden_layer_size = 256
num_layers = 3
num_encoder_layers = num_layers
num_decoder_layers = num_layers
dropout = 0.2
cell_type = 'RNN'

config = {
    "num_encoders" : 3,
    "cell": "RNN",
    "embed_size": 512,
    "dropout": 0.2,
    "num_decoders" : 3,
    "hidden_layer_size":256
}

# TRAIN
model, encoder_layers, decoder_layers = transliteration(config)

# COMPILE
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# FIT
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    shuffle = True,
    # validation_data= ([encoder_val_input_data, decoder_val_input_data], decoder_val_target_data)
)

Epoch 1/7
458/458 [==============================] - 93s 186ms/step - loss: 0.9810 - accuracy: 0.3468
Epoch 2/7
458/458 [==============================] - 85s 185ms/step - loss: 0.6686 - accuracy: 0.5211
Epoch 3/7
458/458 [==============================] - 84s 183ms/step - loss: 0.5576 - accuracy: 0.5849
Epoch 4/7
458/458 [==============================] - 84s 183ms/step - loss: 0.4990 - accuracy: 0.6213
Epoch 5/7
458/458 [==============================] - 83s 182ms/step - loss: 0.4597 - accuracy: 0.6444
Epoch 6/7
458/458 [==============================] - 83s 182ms/step - loss: 0.4337 - accuracy: 0.6607
Epoch 7/7
458/458 [==============================] - 83s 182ms/step - loss: 0.4156 - accuracy: 0.6713


In [ ]:
# config = {
#     "num_encoders" : 1,
#     "cell": "RNN",
#     "embed_size": 32,
#     "dropout": 0.2,
#     "num_decoders" : 1,
#     "hidden_layer_size": 16
# }

# model, enc_layers, dec_layers = transliteration(config)

In [ ]:
# model.compile(
#     optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
# )

# model.fit(
#     [encoder_input_data, decoder_input_data],
#     decoder_target_data,
#     batch_size = 64,
#     epochs = 10
# )

# model.save("s2s")

In [23]:
def inferencing(model,num_encoder_layers,num_decoder_layers,encoder_layers,decoder_layers,cell_type, hidden_layer_size):
    
    # ENCODER MODEL RECONSTRUCTION 
    encoder_inputs = model.input[0]  # input_1
    encoder_states = []
    enc_emb = model.layers[2]     # embedding 1
    encoder_outputs = enc_emb(encoder_inputs)

    if cell_type == 'RNN' or cell_type =="GRU":
        for i in range(num_encoder_layers):
            encoder_outputs, state_h_enc = encoder_layers[i](encoder_outputs)
            encoder_states += [state_h_enc] 
    else:
        for i in range(num_encoder_layers):
            encoder_outputs, state_h_enc, state_c_enc = encoder_layers[i](encoder_outputs)
            encoder_states += [state_h_enc, state_c_enc]   

    encoder_model = Model(encoder_inputs, encoder_states + [encoder_outputs])


    # DECODER MODEL RECONSTRUCTION
    input_names = [["input_100","input_101"],["input_102","input_103"],["input_104","input_105"],"input_106"]

    decoder_inputs = model.input[1]       # input_2
    decoder_embedding = model.layers[3]   # embedding 2
    decoder_outputs = decoder_embedding(decoder_inputs)
    decoder_states = []
    decoder_states_inputs = []
    
    if cell_type == 'RNN' or cell_type =="GRU":
        for i in range(num_decoder_layers):
            decoder_states_inputs += [Input(shape=(hidden_layer_size,), name=input_names[i][0])]
        for i in range(num_decoder_layers):
            decoder_outputs, state_h_dec = decoder_layers[i](decoder_outputs, initial_state=decoder_states_inputs[i])
            decoder_states += [state_h_dec]
    else:
        for i in range(num_decoder_layers):
            decoder_states_inputs += [Input(shape=(hidden_layer_size,), name=input_names[i][0]), Input(shape=(hidden_layer_size,), name=input_names[i][1])]
        j = 0
        for i in range(num_decoder_layers):
            decoder_outputs, state_h_dec, state_c_dec = decoder_layers[i](decoder_outputs, initial_state=decoder_states_inputs[i+j:i+j+2])
            decoder_states += [state_h_dec , state_c_dec]
            j += 1

    decoder_dense = model.layers[4+2*num_encoder_layers]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    return encoder_model, decoder_model

In [24]:
def decode_sequence(input_seq,encoder_model,decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    states_value = states_value[:-1]
    target_seq = np.zeros((1, 1)) 
    target_seq[0, 0] = target_token_index["\t"]
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        dec_ip = [target_seq]+states_value
        output_tokens = decoder_model.predict(dec_ip)
        sampled_token_index = np.argmax(output_tokens[0][0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = output_tokens[1:]
        
    return decoded_sentence

In [25]:
encoder_model, decoder_model = inferencing(model, num_encoder_layers, num_decoder_layers, encoder_layers, decoder_layers, cell_type, hidden_layer_size)
correct = 0
n = val_target.shape[0]
for i in range(20):
    inputs = encoder_val_input_data[i:i+1]
    output = decode_sequence(inputs,encoder_model, decoder_model)
    print("-")
    print("Input sentence:", val_input[i])
    print("Decoded sentence:", output)

-
Input sentence: amka
Decoded sentence: ాంక్

-
Input sentence: anka
Decoded sentence: ాంక్

-
Input sentence: amkam
Decoded sentence: ాంకం

-
Input sentence: ankam
Decoded sentence: ాంకం

-
Input sentence: amgiikarimcha
Decoded sentence: ాంకిష్మరించ

-
Input sentence: angeekarincha
Decoded sentence: ాంకేకరించ్

-
Input sentence: angeekarinchadam
Decoded sentence: ాన్కప్రేకించంగాలాన్రం
-
Input sentence: angikarinchadam
Decoded sentence: ాంగికరిక్షంగా

-
Input sentence: angeekarinchadaniki
Decoded sentence: ాన్సప్రేనికమించి

-
Input sentence: angikarinchadaniki
Decoded sentence: ాంగివార్మటనింది

-
Input sentence: angeekarinchani
Decoded sentence: ాన్కప్రించినపి

-
Input sentence: angikarinchani
Decoded sentence: ాంగికరించిన

-
Input sentence: angeekarinchaaru
Decoded sentence: ాంకీకరించారు

-
Input sentence: angeekarincharu
Decoded sentence: ాన్నేసరించారు

-
Input sentence: angikarinchaaru
Decoded sentence: ాంగికరించారు

-
Input sentence: angeekarinchidi
Decoded sentence: ాన్సిపోరుంటిన

In [ ]:
# def inference(model, num_encoders, num_decoders, enc_layers, dec_layers, cell, hidden_layer_size):

#     enc_in = model.input[0]
#     EmbeddingLayer = model.layers[2]    
#     enc_out = EmbeddingLayer(enc_in)
#     #enc_out = enc_in

#     enc_states = []
#     if cell == 'RNN':
#         for i in range(num_encoders):
#             enc_out, state_h = enc_layers[i](enc_out)
#             enc_states += [state_h] 
#     elif cell == 'LSTM':
#         for i in range(num_encoders):
#             enc_out, state_h, state_c = enc_layers[i](enc_out)
#             enc_states += [state_h, state_c]   
#     elif cell == 'RNN':
#         for i in range(num_encoders):
#             enc_out, state_h = enc_layers[i](enc_out)
#             enc_states += [state_h] 

#     encoder = Model(enc_in, enc_states + [enc_out])


#     dec_in = model.input[1]    
#     EmbeddingLayer = model.layers[3] 
#     dec_out = EmbeddingLayer(dec_in)
#     #dec_out = dec_in

#     dec_states = []
#     dec_initial_states = []
    
#     if cell == 'RNN' :
#         for i in range(num_decoders):
#             dec_initial_states += [Input(shape=(hidden_layer_size,))]
#             dec_out, state_h = dec_layers[i](dec_out, initial_state = dec_initial_states[i])
#             dec_states += [state_h]
#     elif cell == "LSTM":
#         j=0
#         for i in range(num_decoders):
#             dec_initial_states += [Input(shape=(hidden_layer_size, )) , Input(shape=(hidden_layer_size, ))]
#             dec_out, state_h, state_c = dec_layers[i](dec_out, initial_state=dec_initial_states[i+j:i+j+2])
#             dec_states += [state_h , state_c]
#             j += 1
#     elif cell == "GRU":
#         for i in range(num_decoders):
#             dec_initial_states += [Input(shape=(hidden_layer_size,))]
#             dec_out, state_h = dec_layers[i](dec_out, initial_state = dec_initial_states[i])
#             dec_states += [state_h]

#     dec_dense = model.layers[4 + 2*num_encoders]
#     dec_out = dec_dense(dec_out)
#     decoder = Model([dec_in] + dec_initial_states, [dec_out] + dec_states)

#     return encoder, decoder 

In [ ]:
# encoder, decoder = inference(model, 1, 1,  enc_layers, dec_layers, "RNN", 16)

In [ ]:
# def decode_sequence(input_seq,encoder_model,decoder_model):
#     # Encode the input as state vectors.
#     states_value = encoder_model.predict(input_seq)
#     states_value = states_value[:-1]
#     target_seq = np.zeros((1, 1)) 
#     target_seq[0, 0] = target_token_index["\t"]
#     stop_condition = False
#     decoded_sentence = ""
#     while not stop_condition:
#         dec_ip = [target_seq]+states_value
#         output_tokens = decoder_model.predict(dec_ip)
#         sampled_token_index = np.argmax(output_tokens[0][0, -1, :])
#         sampled_char = reverse_target_char_index[sampled_token_index]
#         decoded_sentence += sampled_char
#         if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
#             stop_condition = True

#         target_seq = np.zeros((1, 1))
#         target_seq[0, 0] = sampled_token_index
#         states_value = output_tokens[1:]
        
#     return decoded_sentence

In [ ]:
# def decode_sequence(input_seq, encoder, decoder):

#     states_value = encoder.predict(input_seq)
#     states_value = states_value[:-1]

#     target_seq = np.zeros((1, 1)) 
#     target_seq[0, 0] = target_token_index["\t"]

#     print(input_seq)

#     stop_condition = False
#     decoded_sentence = ""
#     while not stop_condition:
#         output_tokens = decoder.predict([target_seq] + states_value)
#         sampled_token_index = np.argmax(output_tokens[0][0, -1, :])
#         sampled_char = reverse_target_char_index[sampled_token_index]
#         decoded_sentence += sampled_char

#         if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
#             stop_condition = True

#         target_seq = np.zeros((1, 1))
#         target_seq[0, 0] = sampled_token_index
        
#         states_value = output_tokens[1:]
        
#     return decoded_sentence

In [ ]:
input_seq = encoder_input_data[0 :  1]
print(input_seq)

[[ 0. 12. 10.  8. 19.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.]]


In [ ]:
encoder.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 25)]              0         
                                                                 
 embedding (Embedding)       (None, 25, 32)            832       
                                                                 
 simple_rnn_2 (SimpleRNN)    [(None, 25, 16),          784       
                              (None, 16)]                        
                                                                 
Total params: 1,616
Trainable params: 1,616
Non-trainable params: 0
_________________________________________________________________


In [ ]:
decode_sentence = decode_sequence(input_seq, encoder, decoder)

In [ ]:
print(decode_sentence)

In [ ]:
print(input_tokens)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
encoder, decoder = inference(model, 1, 1,  enc_layers, dec_layers, "RNN", 16)
correct = 0
n = val_target.shape[0]
for i in range(20):
    inputs = encoder_val_input_data[i:i+1]
    output = decode_sequence(inputs,encoder, decoder)
    print("-")
    print("Input sentence:", input[i])
    print("Decoded sentence:", output)

-
Input sentence: amkita
Decoded sentence: 

-
Input sentence: ankita
Decoded sentence: 

-
Input sentence: ankitha
Decoded sentence: 

-
Input sentence: ankitam
Decoded sentence: 

-
Input sentence: ankitham
Decoded sentence: 

-
Input sentence: ankitabaavam
Decoded sentence: 

-
Input sentence: ankithabhavam
Decoded sentence: 

-
Input sentence: ankatamicchaadu
Decoded sentence: 

-
Input sentence: ankitamicchadu
Decoded sentence: 

-
Input sentence: ankitamichhaadu
Decoded sentence: 

-
Input sentence: ankithamicchaadu
Decoded sentence: 

-
Input sentence: ankithamichhaadu
Decoded sentence: 

-
Input sentence: amkithamichaaru
Decoded sentence: 

-
Input sentence: ankithamichaaru
Decoded sentence: 

-
Input sentence: ankithamicharu
Decoded sentence: 

-
Input sentence: ankithamaina
Decoded sentence: 

-
Input sentence: ankusham
Decoded sentence: 

-
Input sentence: amkelu
Decoded sentence: 

-
Input sentence: amkeylu
Decoded sentence: 

-
Input sentence: ankelu
Decoded sentence: 



In [ ]:
# Enter the project and the entity name for wandb sweeps
project_name=""
entity_name=""


In [ ]:
def train():
  default_config={
      "cell": "RNN",
      "embed_size":256,
      "dropout":0.2,
      "num_encoders": 1,
      "num_decoders": 1,
      "hidden_layer_size":128,
      "epochs": 1,
      "batch_size": 64
  }

  wandb.init(config=default_config,project=project_name,entity=entity_name)
  config=wandb.config
  wandb.run.name=(
      str(config.cell)
      +"_"+str(config.embed_size)
      +"_"+str(config.dropout)
      +"_"+str(config.num_encoders)
      +"_"+str(config.num_decoders)
      +"_"+str(config.hidden_layer_size)
      +"_"+str(config.epochs)
      +"_"+str(config.batch_size)
  )

  wandb.run.save()

  model=transliteration(config,inp_token_index,tgt_token_index)
  
  model.compile(
      optimizer="adam",
      loss="categorical_crossentropy",
      metrics=["accuracy"]
  )
  earlystopping = EarlyStopping(
          monitor="val_accuracy", min_delta=0.01, patience=5, verbose=2, mode="auto"
      )

  model.fit(
      [e_in,d_in],
      d_t,
      batch_size=config.batch_size,
      epochs=config.epochs,
      callbacks=[earlystopping,WandbCallback()]
  )

  wandb.finish()
  

In [ ]:
def sweeper(project_name,entity_name)
  sweep_configuration={
      "method": "bayes",
      "metric": {"name": "val_accuracy", "goal": "maximize"},
      "parameters":{
          "cell":{"values":["RNN","GRU","LSTM"]},
          "embed_size":{"values":[16,32,64,256]},
          "hidden_layer_size":{"values":[16,32,64,256]},
          "num_encoders":{"values":[1,2,3]},
          "num_decoders":{"values":[1,2,3]},
          "dropout":{"values":[0.2,0.3]},
          "epochs":{"values":[5,10,15,20]},
          "batch_size":{"values":[32,64]}
      }
  }

  sweep_id=wandb.sweep(sweep_configuration,project=project_name,entity=entity_name)
  wandb.agent(sweep_id,train)


In [ ]:
# Running the sweeps for hyperparameter tuning
sweeper(project_name,entity_name)